In [ ]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:

# Ensure the script runs properly on Windows
if __name__ == '__main__':
    # Step 1: Load the YOLOv8 model
    print("Loading YOLOv8 model...")
    try:
        model = YOLO('yolov8n.pt')
    except Exception as e:
        print(f"Error loading model: {e}")
        exit(1)  # Load a pretrained YOLOv8n model (you can use 'yolov8s', 'yolov8m', etc.)

    # Step 2: Train the model
    print("Starting training...")
    data_path="/content/drive/MyDrive/CV_project/moroccan palates.v1i.yolov8/data/data.yaml"
    results = model.train(
        data=data_path,  # Path to your dataset configuration file (from Roboflow)
        epochs=100,  # Number of epochs (adjust based on your dataset size and complexity)
        batch=16,  # Batch size (adjust based on your GPU memory)
        imgsz=640,  # Image size (adjust based on your dataset)
        device='0',  # Use GPU (set to 'cpu' if no GPU is available)
        name='yolov8_license_plate',  # Name of the training run
        save=True,  # Save the trained model
        plots=True,  # Generate and save training plots (e.g., loss curves, metrics)
        verbose=True  # Print detailed training logs
    )

    # Step 3: Save training metrics and graphs
    print("Saving training metrics and graphs...")
    results_dir = "/content/drive/MyDrive/CV_project/moroccan palates.v1i.yolov8/ML/Train" # Path to the results directory

    # Load training results
    metrics = pd.read_csv(os.path.join(results_dir, 'results.csv'))  # Load training metrics

    # Plot and save loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(metrics['epoch'], metrics['train/box_loss'], label='Train Box Loss')
    plt.plot(metrics['epoch'], metrics['val/box_loss'], label='Validation Box Loss')
    plt.plot(metrics['epoch'], metrics['train/cls_loss'], label='Train Class Loss')
    plt.plot(metrics['epoch'], metrics['val/cls_loss'], label='Validation Class Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Curves')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(results_dir, 'loss_curves.png'))  # Save the plot
    plt.close()

    # Plot and save precision-recall curves
    plt.figure(figsize=(10, 6))
    plt.plot(metrics['epoch'], metrics['metrics/precision'], label='Precision')
    plt.plot(metrics['epoch'], metrics['metrics/recall'], label='Recall')
    plt.plot(metrics['epoch'], metrics['metrics/mAP50'], label='mAP@0.5')
    plt.plot(metrics['epoch'], metrics['metrics/mAP50-95'], label='mAP@0.5:0.95')
    plt.xlabel('Epoch')
    plt.ylabel('Metric Value')
    plt.title('Precision, Recall, and mAP Curves')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(results_dir, 'metrics_curves.png'))  # Save the plot
    plt.close()

    # Step 4: Save the trained model and results
    print("Saving the trained model and results...")
    model.export(format='onnx')  # Export the model to ONNX format (optional)
    print(f"Training results and model saved in: {results_dir}")

    # Step 5: Generate a summary report
    print("Generating a summary report...")
    report = f"""
    Training Summary:
    - Model: YOLOv8n
    - Dataset: License Plate Detection
    - Epochs: 50
    - Batch Size: 16
    - Image Size: 640x640
    - Final mAP@0.5: {metrics['metrics/mAP50'].iloc[-1]:.4f}
    - Final mAP@0.5:0.95: {metrics['metrics/mAP50-95'].iloc[-1]:.4f}
    - Final Precision: {metrics['metrics/precision'].iloc[-1]:.4f}
    - Final Recall: {metrics['metrics/recall'].iloc[-1]:.4f}
    - Training Directory: {results_dir}
    """
    print(report)
    with open(os.path.join(results_dir, 'training_summary.txt'), 'w') as f:
        f.write(report)

    print("Training and evaluation complete!")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
